In [ ]:
from ultralytics import YOLO
import cv2
import torch

In [ ]:
# Carregar o modelo YOLOv8 pré-treinado
model = YOLO("yolov8n.pt")  # Use "yolov8l.pt" para mais precisão

# Captura de vídeo da webcam
cap = cv2.VideoCapture(0)

# ROI (Region of Interest) para detecção de objetos
roi_start = (100, 0)  # Coordenadas do canto superior esquerdo da área
roi_end = (400, 500)    # Coordenadas do canto inferior direito da área

# Inicializar gravação de vídeo
video_writer = None
recording = False

# Definir limite de confiança
confidence_threshold = 0.80

In [ ]:
while True:
    # Ler frame da webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar a detecção de objetos com YOLOv8
    results = model(frame)
    
    # Filtrar detecções para encontrar apenas pessoas com confiança >= 0.80
    for result in results:
        for box in result.boxes:
            if box.cls == 0:  # Classe 0 é 'pessoa' no COCO dataset
                confidence = box.conf.item()
                
                if confidence >= confidence_threshold:
                    x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                    
                    # Desenhar o retângulo ao redor da pessoa detectada
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    cv2.putText(frame, f"Pessoa {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

                    # Verificar se a pessoa está dentro da área de interesse
                    if (roi_start[0] < x1 < roi_end[0]) and (roi_start[1] < y1 < roi_end[1]):
                        cv2.putText(frame, "Pessoa detectada na area!", (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                        
                        # Iniciar gravação se não estiver gravando
                        if not recording:
                            fourcc = cv2.VideoWriter_fourcc(*'XVID')
                            video_writer = cv2.VideoWriter('detected.avi', fourcc, 20.0, (frame.shape[1], frame.shape[0]))
                            recording = True

    # Desenhar a área de interesse no frame
    cv2.rectangle(frame, roi_start, roi_end, (0, 255, 0), 2)
    
    # Gravar o frame se estiver gravando
    if recording:
        video_writer.write(frame)

    # Mostrar frame
    cv2.imshow("Deteccao de Pessoas", frame)

    # Parar se a tecla 'q' for pressionada
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar a captura e fechar janelas
cap.release()
if recording:
    video_writer.release()
cv2.destroyAllWindows()